In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")
    f.write("kubeflow-katib==0.12.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [16]:
## import required pkgs

import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import ApiClient
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1EarlyStoppingSpec
from kubeflow.katib import V1beta1EarlyStoppingSetting
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1FeasibleSpace
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec

In [17]:
## define katib objective, stopping criteria, and parameter spaces

experiment_name = "median-stop-lstm"
experiment_namespace = "kubeflow-user-example-com"

# Trial count specification.
max_trial_count = 4
max_failed_trial_count = 2
parallel_trial_count = 1

# Objective specification.
objective=V1beta1ObjectiveSpec(
    type="minimize",
    goal= 5,
    objective_metric_name="Validation-MAE",
    additional_metric_names=[
        "Training-MAE"
    ]
)

# Algorithm specification.
algorithm=V1beta1AlgorithmSpec(
    algorithm_name="random",
)

# Early Stopping specification.
early_stopping=V1beta1EarlyStoppingSpec(
    algorithm_name="medianstop",
    algorithm_settings=[
        V1beta1EarlyStoppingSetting(
            name="min_trials_required",
            value="2"
        )
    ]
)


# Experiment search space.
# In this example we tune learning rate, number of layer and optimizer.
# Learning rate has bad feasible space to show more early stopped Trials.
parameters=[
    V1beta1ParameterSpec(
        name="n",
        parameter_type="int",
        feasible_space=V1beta1FeasibleSpace(
            min="2",
            max="7"
        ),
    ),
    V1beta1ParameterSpec(
        name="lr",
        parameter_type="double",
        feasible_space=V1beta1FeasibleSpace(
            min="0.01",
            max="0.5"
        ),
    ),
#     V1beta1ParameterSpec(
#         name="epochs",
#         parameter_type="int",
#         feasible_space=V1beta1FeasibleSpace(
#             min="100",
#             max="200"
#         ),
#     ),
#     V1beta1ParameterSpec(
#         name="lastnyears",
#         parameter_type="int",
#         feasible_space=V1beta1FeasibleSpace(
#             min="1",
#             max="2"
#         ),
#     ),
]

In [18]:
## define trial templates

trial_spec={
    "apiVersion": "batch/v1",
    "kind": "Job",
    "spec": {
        "template": {
            "metadata": {
                "annotations": {
                     "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "training-container",
                        "image": "docker.io/footprintai/demo-strockpricing-estimator:latest",
                        "command": [
                            "python3",
                            "/app/build-model.py",
                            "--n=${trialParameters.windowSize}",
                            "--learning-rate=${trialParameters.learningRate}"
#                             "--epochs=${trialParameters.epochs}",
#                             "--lastnyears=${trialParameters.lastNYears}",
                        ]
                    }
                ],
                "restartPolicy": "Never"
            }
        }
    }
}

# Configure parameters for the Trial template.
# We set the retain parameter to "True" to not clean-up the Trial Job's Kubernetes Pods.
trial_template=V1beta1TrialTemplate(
    retain=True,
    primary_container_name="training-container",
    trial_parameters=[
        V1beta1TrialParameterSpec(
            name="windowSize",
            description="window size",
            reference="n"
        ),
        V1beta1TrialParameterSpec(
            name="learningRate",
            description="learning rate",
            reference="lr"
        ),
#         V1beta1TrialParameterSpec(
#             name="epochs",
#             description="number of runs",
#             reference="epochs"
#         ),
#         V1beta1TrialParameterSpec(
#             name="lastNYears",
#             description="trace back to last n years data",
#             reference="lastnyears"
#         ),
    ],
    trial_spec=trial_spec
)

In [19]:
## create experiment resources

experiment_spec=V1beta1ExperimentSpec(
    max_trial_count=max_trial_count,
    max_failed_trial_count=max_failed_trial_count,
    parallel_trial_count=parallel_trial_count,
    objective=objective,
    algorithm=algorithm,
    early_stopping=early_stopping,
    parameters=parameters,
    trial_template=trial_template
)

In [20]:
## Create pipeline components

katib_experiment_launcher_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml")

@dsl.pipeline(
    name="Launch Katib early stopping Experiment",
    description="An example to launch Katib Experiment with early stopping"
)

def median_stop():
    
    # Katib launcher component.
    # Experiment Spec should be serialized to a valid Kubernetes object.
    op = katib_experiment_launcher_op(
        experiment_name=experiment_name,
        experiment_namespace=experiment_namespace,
        experiment_spec=ApiClient().sanitize_for_serialization(experiment_spec),
        experiment_timeout_minutes=60,
        delete_finished_experiment=False)
    
    # Output container to print the results.
    op_out = dsl.ContainerOp(
        name="best-hp",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["echo Best HyperParameters: %s" % op.output],
    )

In [21]:
kfp.compiler.Compiler().compile(median_stop, 'helloworld.zip')